In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/data-science-bowl-2019/test.csv
/kaggle/input/data-science-bowl-2019/specs.csv
/kaggle/input/data-science-bowl-2019/train.csv
/kaggle/input/data-science-bowl-2019/train_labels.csv
/kaggle/input/data-science-bowl-2019/sample_submission.csv
/kaggle/input/temp-ds-bowl-2019/labels_created
/kaggle/input/temp-ds-bowl-2019/df_counter_ini
/kaggle/input/temp-ds-bowl-2019/test_all_users.pkl
/kaggle/input/temp-ds-bowl-2019/all_users.pkl


In [2]:
import IPython

def display(*dfs):
    for df in dfs:
        IPython.display.display(df)
        
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
%time df = pd.read_csv('/kaggle/input/data-science-bowl-2019/train.csv', engine='c')
# labels = pd.read_csv('/kaggle/input/data-science-bowl-2019/train_labels.csv')
df_test = pd.read_csv('/kaggle/input/data-science-bowl-2019/test.csv')
specs = pd.read_csv('/kaggle/input/data-science-bowl-2019/specs.csv')

df = reduce_mem_usage(df)
# labels = reduce_mem_usage(labels)
df_test = reduce_mem_usage(df_test)
specs = reduce_mem_usage(specs)

CPU times: user 1min 6s, sys: 11.6 s, total: 1min 18s
Wall time: 1min 18s
Mem. usage decreased to 778.73 Mb (18.2% reduction)
Mem. usage decreased to 79.40 Mb (18.2% reduction)
Mem. usage decreased to  0.01 Mb (0.0% reduction)


In [4]:
# df_counter = pd.read_feather('/kaggle/input/temp-ds-bowl-2019/df_counter_ini')
# df_counter = pd.read_feather('/kaggle/input/temp-ds-bowl-2019/df_counter')
labels = pd.read_feather('/kaggle/input/temp-ds-bowl-2019/labels_created')

/opt/conda/lib/python3.6/site-packages/pandas/io/feather_format.py:117: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_feather(path, columns=columns, nthreads=int_use_threads)
/opt/conda/lib/python3.6/site-packages/pyarrow/pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [5]:
# all_users = pd.read_pickle('/kaggle/input/temp-ds-bowl-2019/all_users.pkl')
# all_users.installation_id.unique().shape, labels.installation_id.unique().shape

# test_all_users = pd.read_pickle('/kaggle/input/temp-ds-bowl-2019/test_all_users.pkl')
# test_all_users.installation_id.unique().shape, test_all_users.shape

In [6]:
import gc
gc.collect()

22

# Part I - Create all_users
## One user

In [7]:
#user_id, user = next(iter(df.groupby('installation_id')))
user_id = 'f10d8174'#'ea49ea9c'#'29d1aaee' # '3f0dca37'
user = df[df.installation_id == user_id].copy()

totall_attempts = labels[labels.installation_id == user_id].total_attempts.sum()
print(totall_attempts)

totall_assessments  = labels[labels.installation_id == user_id].shape[0]
totall_assessments

67


38

In [8]:
labels[labels.installation_id == user_id]

,game_session,world,type,timestamp,num_correct,total_attempts,installation_id,title,accuracy,num_incorrect,accuracy_group
201,03282df6833bb705,TREETOPCITY,Assessment,2019-08-02 15:06:40.664000+00:00,1,1,f10d8174,Mushroom Sorter (Assessment),1.000000,0,3
286,045000369ce63c29,MAGMAPEAK,Assessment,2019-07-24 15:03:35.017000+00:00,1,1,f10d8174,Cauldron Filler (Assessment),1.000000,0,3
695,0a13d269c0c8b48b,CRYSTALCAVES,Assessment,2019-07-29 20:04:32.052000+00:00,1,1,f10d8174,Cart Balancer (Assessment),1.000000,0,3
813,0baeca09cb52c79a,CRYSTALCAVES,Assessment,2019-08-10 00:16:06.187000+00:00,0,4,f10d8174,Chest Sorter (Assessment),0.000000,4,0
1651,18192e1247fd5063,TREETOPCITY,Assessment,2019-07-29 20:48:43.526000+00:00,1,1,f10d8174,Mushroom Sorter (Assessment),1.000000,0,3
3748,36c0a398d58ebf90,CRYSTALCAVES,Assessment,2019-07-29 20:13:20.236000+00:00,0,3,f10d8174,Chest Sorter (Assessment),0.000000,3,0
4053,3b7b5617b5ac367c,TREETOPCITY,Assessment,2019-07-25 16:36:45.631000+00:00,1,3,f10d8174,Bird Measurer (Assessment),0.333333,2,1
4305,3f47728273d128a5,CRYSTALCAVES,Assessment,2019-08-11 02:19:56.720000+00:00,1,1,f10d8174,Cart Balancer (Assessment),1.000000,0,3
4475,41adc9bd8666a6c1,CRYSTALCAVES,Assessment,2019-07-29 20:30:18.437000+00:00,1,1,f10d8174,Cart Balancer (Assessment),1.000000,0,3
4889,475382fc8aa87fab,CRYSTALCAVES,Assessment,2019-10-14 20:01:51.232000+00:00,1,1,f10d8174,Cart Balancer (Assessment),1.000000,0,3


In [9]:
user['timestamp'] = pd.to_datetime(user['timestamp'])
print(user.shape)
user.sort_values(by='timestamp').head()

user['attempt'] = ((user.type == 'Assessment') &
                       (((user.event_code == 4100) & (user.title != 'Bird Measurer (Assessment)')) |
                        ((user.event_code == 4110)&(user.title == 'Bird Measurer (Assessment)')) )
                   ).astype('int8')
#attempt_idx = user[user.attempt == 1].index
assert user.attempt.sum() == totall_attempts

user['correct'] = 0
user.loc[user.attempt == 1, 'correct'] = user[user.attempt == 1].event_data.str.contains('"correct":true').astype('int8')
#user

(13152, 11)


In [10]:
def current_attempt_save(attempt, user_temp, session):
    temp = user_temp.copy()
    attempt['total_accuracy'] = np.mean(temp['total_accuracy']) \
                                   if isinstance(temp['total_accuracy'], list) else -1
    attempt['total_accuracy_groups'] = np.mean(temp['total_accuracy_groups']) \
                                          if isinstance(temp['total_accuracy_groups'], list) else -1        
    attempt['curr_attempt_world'] = session.world.iloc[0]
    attempt['curr_attempt_title'] = session.title.iloc[0]
    attempt['curr_attempt_start_time'] = session.timestamp.iloc[0]
    attempt['curr_attempt_duration'] = attempt['time_start'] - attempt['curr_attempt_start_time']
        
    return attempt

def current_world_save(attempt, user_temp, world):
    # current WORLD - write all previous info in current world
    features = [x for x in user_temp.keys() if world in str(x)]
    temp = defaultdict(int)
    temp.update({k.replace(world, 'world'):user_temp[k] for k in features})
    f = [x.replace(world, 'world') for x in features if 'unique' in x]
    assert len(f) <= 4
    temp.update({k: len(set(temp[k])) for k in f})
    temp['world_accuracy'] = np.mean(temp['world_accuracy']) \
                             if isinstance(temp['world_accuracy'], list) else -1
    temp['world_accuracy_groups'] = np.mean(temp['world_accuracy_groups']) \
                                    if isinstance(temp['world_accuracy_groups'], list) else -1

    f = [x for x in features if 'game_time' in x]
    temp['world_total_game_time'] = np.sum([user_temp[x] for x in f])
    f = [x for x in features if 'event_count' in x]
    temp['world_total_event_count'] = np.sum([user_temp[x] for x in f])
    f = [x for x in features if 'n_game_session' in x]
    temp['world_total_n_game_session'] = np.sum([user_temp[x] for x in f])
    attempt.update(temp)
    
    return attempt


def current_title_save(attempt, user_temp, title):
    # current TITLE - write all previous info in attempts with the same title
    features = [x for x in user_temp.keys() if title in str(x)]
    temp = defaultdict(int)
    temp.update({k.replace(title, 'title'):user_temp[k] for k in features})
    temp['title_accuracy'] = np.mean(temp['title_accuracy']) \
                             if isinstance(temp['title_accuracy'], list) else -1
    temp['title_accuracy_groups'] = np.mean(temp['title_accuracy_groups']) \
                                    if isinstance(temp['title_accuracy_groups'], list) else -1
    attempt.update(temp)
    
    return attempt

def create_sample(user_data, user_temp, session):
    attempt = user_data.copy()
    title = session.title.iloc[0]
    world = session.world.iloc[0]
    
    attempt = current_attempt_save(attempt, user_temp, session)
    attempt = current_world_save(attempt, user_temp, world)
    attempt = current_title_save(attempt, user_temp, title)

    # to labels
    attempt['n_correct'] = session.correct.sum()
    attempt['n_incorrect'] = session.attempt.sum() - attempt['n_correct']

    # TOTAL info
    for t in ['Clip', 'Game', 'Activity', 'Assessment']:
        attempt['total_' + t +'_unique'] = len(set(attempt['total_' + t +'_unique'])) \
                                                if isinstance(attempt['total_' + t +'_unique'], list) \
                                                and len(attempt['total_' + t +'_unique']) >0 else 0

    return attempt

In [11]:
accuracy_to_group = {1: 3, 0.5: 2, 0: 0}

def get_common_data(session, user_data, user_temp, user_results, test=False):
    """
    params: 
        session: DataFrame - one session form groupby('game_session')
        user_data: dict - dict with data info about user. Accumulate all infor for user though attempts.
    return:
        user_results: list - list of dictionaries, one for each attempt of one user
    """
#     display(session)
    
    data_type = session.type.iloc[0]
    world = session.world.iloc[0]
    template = world +'_'+ data_type
    user_temp[template+'_unique'] = user_temp.get(template+'_unique', [])
    user_data['total_' + data_type+'_unique'] = user_data.get('total_' + data_type+'_unique', [])
    
    if data_type == 'Assessment':
        if session.attempt.sum() >= 1 or test:  
            # create sample for train set
            attempt = create_sample(user_data, user_temp, session)
            user_results.append(attempt)

        #  ======= Save info for future attempt ====== title = session.title.iloc[0]       
        # current title - CALCULATE (accumulate) all previous info in attempts with the same title
        n_correct = session.correct.sum()
        n_incorrect = session.attempt.sum() - n_correct
        
        title = session.title.iloc[0]
        user_temp[title + '_n_assessments'] += 1
        user_temp[title + '_n_assess_with_attempt'] += 1 if session.attempt.sum() > 0 else 0
        user_temp[title + '_n_attempts'] += session.attempt.sum()
        user_temp[title + '_n_correct'] += n_correct
        user_temp[title + '_n_incorrect'] = n_incorrect
        #user_temp[title + '_n_attempts'] - user_temp[title + '_n_correct']
        user_temp[title + '_n_unfinished'] += 0  if session.attempt.sum() > 0 else 1
        
        #print(title, user_temp[title + '_n_assessments'], user_temp[title + '_n_assess_with_attempt'] , user_temp[title + '_n_unfinished'] )
        if session.attempt.sum() != 0:
            accuracy = session.correct.sum()/ session.attempt.sum()
            user_temp[title + '_accuracy'] = user_temp.get(title+'_accuracy_groups', []) +[accuracy]
            group = accuracy_to_group.get(accuracy, 1)
            user_temp[title+'_accuracy_groups'] = user_temp.get(title+'_accuracy_groups', []) +[group]
#         else:
#             print('# Attemps == 0', session.installation_id)
        user_temp[title + '_event_count'] += session.shape[0]
        user_temp[title + '_game_time'] += session.game_time.max()
        
        # Currect attempts - to future
        user_temp[world+'_n_correct'] += n_correct
        user_temp[world+'_n_incorrect'] += n_incorrect
        user_data['total_n_correct'] += n_correct
        user_data['total_n_incorrect'] += n_incorrect
        
        if n_correct + n_incorrect != 0:
            accuracy = n_correct/(n_correct+n_incorrect)
            group = accuracy_to_group.get(accuracy, 1)
            
            user_temp[world+'_accuracy'] = user_temp.get(world+'_accuracy', []) +[accuracy]
            user_temp[world+'_accuracy_groups'] = user_temp.get(world+'_accuracy_groups', []) +[group]
            user_temp['total_accuracy'] = user_temp.get('total_accuracy', []) +[accuracy]
            user_temp['total_accuracy_groups'] = user_temp.get('total_accuracy_groups', []) +[group]        
        
    event_code_counts = session['event_code'].value_counts()
    for i, j in zip(event_code_counts.index, event_code_counts.values):
        user_data[i] = user_data.get(i,0) + j
        
    # add common info, include attempt info for next study
    user_temp[template+'_unique'].append(session.title.iloc[0])
    user_temp[template+'_n_game_session'] += 1
    user_temp[template+'_event_count'] += session.shape[0]
    
    template_total = 'total_' + data_type
    user_data[template_total+'_unique'].append(session.title.iloc[0])
    user_data[template_total+'_n_game_session'] += 1
    user_data[template_total+'_event_count'] += session.shape[0]
    
    if data_type != 'Clip':
        user_temp[template+'_game_time'] += session.game_time.max()
        user_data[template_total+'_game_time'] += session.game_time.max()
        user_data['total_game_time'] += session.game_time.max()
        
    user_data['total_' + data_type] += 1
    user_data['total_n_game_session'] += 1
    user_data['total_event_count'] += session.shape[0]
    
        
        
    return user_results


In [12]:
%%time 
user_results = []

from collections import defaultdict
user_data, user_temp = defaultdict(int), defaultdict(int)
user_data['installation_id'] = user.installation_id.iloc[0]
user_data['time_start'] = user.timestamp.iloc[0]

for i, session in user.groupby('game_session', sort=False):
    #print(user_data)
    get_common_data(session, user_data, user_temp, user_results)
    
tt = pd.DataFrame(user_results)

CPU times: user 716 ms, sys: 4 ms, total: 720 ms
Wall time: 720 ms


In [13]:
title_columns = [x for x in tt.columns if 'title' in str(x) and 'acc' not in x]

In [14]:
tt[title_columns]

,curr_attempt_title,title_n_assessments,title_n_assess_with_attempt,title_n_attempts,title_n_correct,title_n_incorrect,title_n_unfinished,title_event_count,title_game_time
0,Cart Balancer (Assessment),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cauldron Filler (Assessment),1.0,0.0,0.0,0.0,0.0,1.0,44.0,55359.0
2,Mushroom Sorter (Assessment),1.0,0.0,0.0,0.0,0.0,1.0,37.0,43418.0
3,Bird Measurer (Assessment),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cart Balancer (Assessment),1.0,1.0,2.0,1.0,1.0,0.0,58.0,62096.0
5,Chest Sorter (Assessment),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Mushroom Sorter (Assessment),2.0,1.0,4.0,1.0,3.0,1.0,101.0,125616.0
7,Bird Measurer (Assessment),1.0,1.0,1.0,1.0,0.0,0.0,49.0,95564.0
8,Cauldron Filler (Assessment),2.0,1.0,1.0,1.0,0.0,1.0,71.0,92376.0
9,Bird Measurer (Assessment),2.0,2.0,6.0,2.0,4.0,0.0,109.0,182713.0


## Total: for all users

### Info for last attemp

In [15]:
def create_all_users(df, test=False):
    global labels
    all_users = pd.DataFrame()
    
    if test: labels = df  # all unique users with attemptions; for test - all users
    for n,user_id in enumerate(labels.installation_id.unique()):
        # print(n, user_id)
        user = df[df.installation_id == user_id].copy()
        user['timestamp'] = pd.to_datetime(user['timestamp'])
        user.sort_values(by='timestamp', inplace=True)

        user['attempt'] = ((user.type == 'Assessment') &
                           (((user.event_code == 4100) & (user.title != 'Bird Measurer (Assessment)')) |
                            ((user.event_code == 4110)&(user.title == 'Bird Measurer (Assessment)')) )
                       ).astype('int8')

        user['correct'] = 0
        user.loc[user.attempt == 1, 'correct'] = user[user.attempt == 1]\
                                                .event_data.str.contains('"correct":true').astype('int8')

        user_results = []

        user_data, user_temp = defaultdict(int), defaultdict(int)
        user_data['installation_id'] = user.installation_id.iloc[0]
        user_data['time_start'] = user.timestamp.iloc[0]
        
        for i, session in user.groupby('game_session', sort=False):
            get_common_data(session, user_data, user_temp, user_results, test=test)

        temp = all_users.shape
        all_users = pd.concat((all_users,pd.DataFrame(user_results)), ignore_index=True, sort=False)
        
        assert all_users.shape > temp, user_id
#         display(all_users)
#         input()
    
    return all_users

In [16]:
all_users = create_all_users(df)

print(all_users.shape, all_users.installation_id.unique().shape)
all_users.columns = all_users.columns.astype(str)
# all_users.to_pickle('all_users.pkl')

(17690, 108) (3614,)


# Creating test data

In [17]:
labels.installation_id.unique().shape, df_test.installation_id.unique().shape

((3614,), (1000,))

In [18]:
test_all_users = create_all_users(df_test, test=True)

# take only last assessment
test_all_users = test_all_users.groupby('installation_id', sort=False, as_index=False)\
                            .agg([lambda x: x.iloc[-1]])
test_all_users.columns = [x[0] for x in test_all_users.columns]
test_all_users.reset_index(inplace=True)

print(test_all_users.shape, test_all_users.installation_id.unique().shape)
test_all_users.columns = test_all_users.columns.astype(str)
# test_all_users.to_pickle('test_all_users.pkl')

(1000, 108) (1000,)


# Part II - preprocess train data
## Features names

In [19]:
c = test_all_users.columns
world_columns = [x for x in c if 'world' in x and 'curr' not in x]
title_columns = [x for x in c if 'title' in x and 'curr' not in x]
current_columns = [x for x in c if 'curr' in x] + ['installation_id', 'time_start', 'n_correct', 'n_incorrect', 
                                                   'total_n_game_session', 'total_event_count', 'total_game_time',
                                                   'total_n_correct', 'total_n_incorrect', 
                                                   'total_accuracy', 'total_accuracy_groups']
total_columns = [x for x in c if 'total' in x and 'world' not in x and x not in current_columns]
event_columns = [x for x in c if x.isdigit()]
all_columns = world_columns+title_columns+current_columns+total_columns+event_columns
assert c.shape[0] == len(all_columns), (c.shape[0] ,len(all_columns))

## Fill NAN

In [20]:
test_all_users.fillna(0., inplace=True)
all_users.fillna(0., inplace=True)

## Work with time

In [21]:
def preprocess_time(test_all_users):
    time_columns = ['time_start', 'curr_attempt_start_time'] 
    for t in time_columns:
        test_all_users[t+'_month'] = test_all_users[t].dt.month
        test_all_users[t+'_day'] = test_all_users[t].dt.day
        test_all_users[t+'_hour'] = test_all_users[t].dt.hour
        test_all_users[t+'_dayofweek'] = test_all_users[t].dt.dayofweek
        test_all_users[t+'_month'] = test_all_users[t].dt.month
        test_all_users[t+'_quarter'] = test_all_users[t].dt.quarter

    t =  'curr_attempt_duration'
    test_all_users[t+'seconds'] = -test_all_users[t].dt.total_seconds()
    test_all_users[t+'hours'] = -test_all_users[t].dt.total_seconds()/60/60
    
    return test_all_users
    
    
def create_means(all_users, temp, t=''):
    if t != '': t ='_' + t
    if t != '_Clip':
        all_users[temp+t+'_game_time_mean'] =  all_users[temp+t+'_game_time']/ \
                                                    all_users[temp+t+'_n_game_session']
        all_users[temp+t+'_event_time_mean'] =  all_users[temp+t+'_game_time']/ \
                                                    all_users[temp+t+'_event_count']
            
    all_users[temp+t+'_event_mean'] = all_users[temp+t+'_event_count']/ \
                                          all_users[temp+t+'_n_game_session']
    return all_users


def preprocess_data(all_users):
    for temp in ['world', 'total']:
        for t in ['Clip', 'Game', 'Assessment', 'Activity', 'total']:
            if temp=='total' and t=='total': continue
            all_users = create_means(all_users, temp, t)
        
        
    all_users['title_game_time_mean'] = all_users['title_game_time']/ all_users['title_n_assessments']                                            
    all_users['title_event_time_mean'] = all_users['title_game_time']/ all_users['title_event_count']                                            
    all_users['title_event_mean'] = all_users['title_event_count']/ all_users['title_n_assessments']
    
    all_users = preprocess_time(all_users)
    all_users = create_means(all_users, temp='total', t='')
        
    all_users.fillna(0., inplace=True)               
        
    return all_users

In [22]:
test_all_users = preprocess_data(test_all_users)
all_users = preprocess_data(all_users)

In [23]:
test_all_users.shape, all_users.shape

((1000, 149), (17690, 149))

In [24]:
c = test_all_users.columns
world_columns = [x for x in c if 'world' in x and 'curr' not in x]
title_columns = [x for x in c if 'title' in x and 'curr' not in x]
current_columns = [x for x in c if 'curr' in x] + ['installation_id', 'time_start', 'n_correct', 'n_incorrect', 
                                                   'total_n_game_session', 'total_event_count', 'total_game_time',
                                                   'total_n_correct', 'total_n_incorrect', 
                                                   'total_accuracy', 'total_accuracy_groups',
                                                   'total_game_time_mean', 'total_event_time_mean', 'total_event_mean',
                                                   'time_start_month', 'time_start_day', 'time_start_hour',
                                                   'time_start_dayofweek', 'time_start_quarter']
total_columns = [x for x in c if 'total' in x and 'world' not in x and x not in current_columns]
event_columns = [x for x in c if x.isdigit()]
all_columns = world_columns+title_columns+current_columns+total_columns+event_columns
assert c.shape[0] == len(all_columns), (c.shape[0] ,len(all_columns))

In [25]:
test_all_users.to_pickle('test_all_users.pkl')
all_users.to_pickle('all_users.pkl')